In [1]:
"""
clone and install PySDM dependencies in Colab
"""
import os, sys
if 'google.colab' in sys.modules:
    %cd /content
    if not os.path.isdir('PySDM'):
        !git clone --depth 1 https://github.com/atmos-cloud-sim-uj/PySDM.git
        %cd PySDM
    else:
        %cd PySDM
        !git pull
    !pip install -r requirements.txt
    !ldconfig

In [2]:
import os, sys
if 'google.colab' in sys.modules:
    %cd /content/PySDM
else:
    sys.path.insert(0, os.path.join(os.getcwd(), '../..'))

In [3]:
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.settings import Settings
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.simulation import Simulation
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.storage import Storage
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.netcdf_exporter import NetCDFExporter
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.demo_plots import _TimeseriesPlot, _ImagePlot

from PySDM_examples.utils.temporary_file import TemporaryFile
from PySDM_examples.utils.widgets import HTML, display
from PySDM_examples.Bartman_et_al_2021.progbar_controller import ProgBarController
import PySDM.products as PySDM_products

from PySDM.physics import si
import os
import numpy as np
from scipy.io.netcdf import netcdf_file
from matplotlib import pyplot, rcParams
from matplotlib.animation import FuncAnimation
from matplotlib.gridspec import GridSpec

/home/slayoo/devel/PySDM/PySDM_examples/Bartman_et_al_2021/../../PySDM/backends/__init__.py:29: UserWarning: CUDA library found but cuInit() failed (error code: 999; message: unknown error)
  warnings.warn(


In [4]:
settings = Settings()

cloud_range = (settings.aerosol_radius_threshold, settings.drizzle_radius_threshold)
products = [
    PySDM_products.AerosolSpecificConcentration(radius_threshold=cloud_range[0]),
    PySDM_products.CloudDropletEffectiveRadius(radius_range=(cloud_range[0], np.inf)),
    PySDM_products.DrizzleConcentration(radius_threshold=cloud_range[1]),
    PySDM_products.WaterMixingRatio(name='qt', description_prefix='total', radius_range=cloud_range)
]

settings.n_sd_per_gridbox = 128 if 'CI' not in os.environ else 32
settings.grid = (32, 32)
settings.dt = 32 * si.second
settings.simulation_time = 1.75 * settings.spin_up_time
settings.output_interval = 1 * si.minute
settings.condensation_rtol_x = 1e-6
settings.condensation_rtol_thd = 5e-7

settings.condensation_dt_cond_range = (.25*si.s, settings.dt)
settings.coalescence_dt_coal_range = settings.condensation_dt_cond_range

settings.mode_1.norm_factor *= 3
settings.mode_2.norm_factor *= 3
settings.spectrum_per_mass_of_dry_air.norm_factor *= 3

storage = Storage()
simulation = Simulation(settings, storage)
simulation.reinit(products)

In [5]:
simulation.run(ProgBarController())

FloatProgress(value=0.0, max=1.0)

In [6]:
temp_file = TemporaryFile('.nc')
exporter = NetCDFExporter(storage, settings, simulation, temp_file.absolute_path)
exporter.run(ProgBarController())

FloatProgress(value=0.0, max=1.0)

In [7]:
default_figsize = rcParams["figure.figsize"]
figsize = (1.75 * default_figsize[0], 3.1* default_figsize[1])
gs = GridSpec(nrows=27, ncols=18)

fig = pyplot.figure(figsize=figsize)
na = gs.nrows-19
nb = gs.nrows-11
nc = gs.nrows-3
axs = (
    fig.add_subplot(gs[:na, 1:gs.ncols//2-1]),
    fig.add_subplot(gs[:na, gs.ncols//2+1:2*gs.ncols//2-1]),

    fig.add_subplot(gs[na:nb, 1:gs.ncols//2-1]),
    fig.add_subplot(gs[na:nb, gs.ncols//2+1:2*gs.ncols//2-1]),
    
    fig.add_subplot(gs[nb:nc, 1:gs.ncols//2-1]),
    fig.add_subplot(gs[nb:nc, gs.ncols//2+1:2*gs.ncols//2-1]),
    
    fig.add_subplot(gs[gs.nrows-3:-1,2:-1])
)
gs.tight_layout(fig)
ncdf = netcdf_file(temp_file.absolute_path, mode='r', mmap=False)

# TODO #419: we should not use products here - all info should be obtained from netCDF
plots = []
for var, cmap in {
    'n_a_mg': 'summer',
    'n_d_cm3': 'bone_r',
    'r_eff': 'ocean_r',
    'qt': 'Blues',
    'dt_cond_min': 'tab20c',
    'dt_coal_min': 'tab20c'
}.items():
    plots.append(
        _ImagePlot(fig, axs[len(plots)], grid=ncdf.grid, size=ncdf.size, product=simulation.products[var], cmap=cmap)
    )
plots.append(_TimeseriesPlot(fig, axs[-1], ncdf.variables['T'][:], show=False))

plots[-1].ax.axvline(ncdf.n_spin_up * ncdf.dt)
plots[-1].ax.set_ylim(0, .001)

interval = 100 #ms
frame_list = np.arange(ncdf.variables['T'].shape[0], dtype=np.int)

def update(frame_num):
    step = frame_num*ncdf.steps_per_output_interval
    
    for i, product in enumerate(('n_a_mg', 'n_d_cm3', 'r_eff', 'qt', 'dt_cond_min', 'dt_coal_min')):
        plots[i].update(ncdf.variables[product][frame_num], step=step)
    
    precip = np.full_like(ncdf.variables['surf_precip'][:], np.nan)
    precip[0:frame_num+1] = ncdf.variables['surf_precip'][0:frame_num+1]
    plots[-1].update(precip)
    
    return (
        plots[0].im, plots[1].im, plots[2].im, 
        plots[3].im, plots[4].im, plots[5].im, 
        plots[-1].timeseries
    )
pyplot.close(fig)

<ipython-input-7-877bf9ecd259>:43: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  frame_list = np.arange(ncdf.variables['T'].shape[0], dtype=np.int)


In [8]:
animation = FuncAnimation(fig, update, frames=frame_list, interval=interval, blit=False)

if 'CI' not in os.environ:
    display(HTML(animation.to_html5_video()))
    file = TemporaryFile('.gif')
    animation.save(file.absolute_path)
    display(file.make_link_widget())

/home/slayoo/devel/PySDM/PySDM_examples/Bartman_et_al_2021/../../PySDM_examples/Arabas_et_al_2015_Figs_8_9/demo_plots.py:68: RuntimeWarning: All-NaN slice encountered
  self.ax.set_title(f"min:{np.nanmin(data): .3g}    max:{np.nanmax(data): .3g}    t/dt:{step: >6}")


HTML(value='<video width="756" height="892" controls autoplay loop>\n  <source type="video/mp4" src="data:vide…

/home/slayoo/devel/PySDM/PySDM_examples/Bartman_et_al_2021/../../PySDM_examples/Arabas_et_al_2015_Figs_8_9/demo_plots.py:68: RuntimeWarning: All-NaN slice encountered
  self.ax.set_title(f"min:{np.nanmin(data): .3g}    max:{np.nanmax(data): .3g}    t/dt:{step: >6}")


HTML(value="<a href='../utils/temporary_files/tmppa13hhjo.gif' target='_blank'>../utils/temporary_files/tmppa1…

In [9]:
# save last frame in vector format
svg_file = TemporaryFile('.svg')
fig.savefig(svg_file.absolute_path)
display(svg_file.make_link_widget())

HTML(value="<a href='../utils/temporary_files/tmpmhgpxas4.svg' target='_blank'>../utils/temporary_files/tmpmhg…